In [1]:
%env TORCH_USE_CUDA_DSA=1
%env CUDA_LAUNCH_BLOCKING=1

env: TORCH_USE_CUDA_DSA=1
env: CUDA_LAUNCH_BLOCKING=1


In [2]:
from typing import Tuple
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import pandas as pd

import numpy as np

import torch

from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Optimizer, Adam
from torch.optim.lr_scheduler import LRScheduler

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [4]:
torch.set_default_dtype(torch.float64)

In [5]:
# df = pd.concat([pd.read_csv("more-elements/more-elements.csv"), pd.read_csv("new-more-data/new-more-data.csv")])
# df

In [6]:
# elems = df["element_1"].unique()
# elem2id = {}
# for i, elem in enumerate(elems):
#     elem2id[elem] = i

# df.fillna(
#     {
#         "element_1": "UNKNOWN",
#         "element_2": "UNKNOWN",
#         "element_3": "UNKNOWN",
#         "element_1_ratio": 0.0,
#         "element_2_ratio": 0.0,
#         "element_3_ratio": 0.0,
#     },
#     inplace=True,
# )

# df.dropna(axis=0, inplace=True)

# elem2id

In [7]:
# df["element_1"] = df["element_1"].apply(
#     lambda x: elem2id[x] if x != "UNKNOWN" else -1
# )
# df["element_2"] = df["element_2"].apply(lambda x: elem2id[x] if x != "UNKNOWN" else -1)
# df["element_3"] = df["element_3"].apply(lambda x: elem2id[x] if x != "UNKNOWN" else -1)
# df

In [8]:
df = pd.read_pickle("half_data.pkl")
df

,0,1,2,3,4,5,6,7,8,9,...,1000,element_1,element_2,element_3,element_1_ratio,element_2_ratio,element_3_ratio,temp,pressure,air_ratio
0,1.402026e-21,1.650584e-21,1.910661e-21,2.264304e-21,2.829955e-21,3.673546e-21,4.745899e-21,5.773241e-21,6.446840e-21,6.741727e-21,...,3.466256e-20,17,23,-1,0.440157,0.559843,0.000000,283.0,0.775,0.0
1,4.102767e-22,4.371429e-22,4.698476e-22,5.105580e-22,5.626400e-22,6.315813e-22,7.266133e-22,8.610945e-22,1.048941e-21,1.323529e-21,...,2.481710e-21,0,19,-1,0.428047,0.571953,0.000000,303.0,0.800,0.6
2,3.610634e-23,2.739156e-23,1.628379e-23,1.195799e-23,1.069667e-23,1.058083e-23,1.121691e-23,1.274356e-23,1.515703e-23,1.882304e-23,...,2.756785e-21,2,11,13,0.342547,0.376666,0.280788,283.0,0.500,0.6
3,1.016487e-22,1.356304e-22,1.894228e-22,3.406116e-22,8.410447e-22,2.039525e-21,1.635235e-21,6.587985e-22,3.087047e-22,2.124933e-22,...,1.240249e-22,6,10,13,0.053299,0.321178,0.625524,293.0,0.900,0.6
4,1.637227e-21,2.113385e-21,2.633061e-21,3.098701e-21,3.509187e-21,3.569827e-21,3.557713e-21,3.356628e-21,3.389955e-21,3.744591e-21,...,3.614986e-23,9,18,24,0.470739,0.251465,0.277796,273.0,0.500,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183548,1.720379e-22,2.933964e-22,4.190668e-22,5.128756e-22,6.537258e-22,6.598291e-22,7.231058e-22,6.818061e-22,8.002978e-22,1.130003e-21,...,8.585612e-23,9,17,23,0.401527,0.454117,0.144356,313.0,0.500,0.6
183549,2.392508e-21,1.998411e-21,1.320957e-21,9.131813e-22,6.948151e-22,5.720024e-22,4.987314e-22,4.533583e-22,4.270046e-22,4.143369e-22,...,2.460996e-21,4,8,16,0.348675,0.389984,0.261341,323.0,1.000,0.6
183550,1.947286e-22,2.092204e-22,2.261748e-22,2.462494e-22,2.703056e-22,2.995569e-22,3.357735e-22,3.814871e-22,4.405433e-22,5.189812e-22,...,1.838447e-23,1,7,14,0.116401,0.430989,0.452610,313.0,0.100,0.6
183551,4.523992e-22,6.372981e-22,1.661855e-21,1.874233e-22,1.226359e-22,1.804612e-22,3.107570e-22,1.559673e-21,3.250324e-21,2.427107e-21,...,1.508687e-21,15,18,23,0.422131,0.441890,0.135979,283.0,0.200,0.3


In [9]:
print(len(df))
df = df[(df[[str(i) for i in range(1001)]] > 0).all(axis=1)]
print(len(df))

183553
183553


In [10]:
len(df[(df["element_1"] == 0) | (df["element_2"] == 0) | (df["element_3"] == 0)]) / len(df)

0.11451733286843582

In [11]:
class CustomSpectraDataset(Dataset):
    def __init__(self, data: pd.DataFrame, device="cuda:0") -> None:
        self.data = data
        self.elements = self.data["element_1"].unique()

        self.spectras = torch.log(
            torch.tensor(df[[str(i) for i in range(1001)]].to_numpy(dtype=np.float64))
        ).to(device)

        self.ratios = torch.tensor(
            df[["element_1_ratio", "element_2_ratio", "element_3_ratio"]].to_numpy(
                dtype=np.float64
            )
        ).to(device)

        self.element_indices = df[["element_1", "element_2", "element_3"]].to_numpy(
            dtype=np.float64
        )

        self.elements_distributions = torch.zeros(
            [len(self.data), len(self.elements)], dtype=torch.float64
        ).to(device)

        for idx in range(len(self.data)):
            indices = self.element_indices[idx, :]
            indices = indices[indices != -1]
            self.elements_distributions[idx, indices] = torch.where(self.ratios[idx][
                range(indices.shape[0])
            ] > 0.0, 1.0, 0.0).double()

    def __len__(self) -> int:
        return len(self.data)

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor]:
        spectra = self.spectras[idx]
        elements_distribution = self.elements_distributions[idx]

        return spectra, elements_distribution

In [12]:
class THzFCNN(nn.Module):
    def __init__(self, n_elements: int) -> None:
        super().__init__()
        self.n_elements = n_elements
        self.net = nn.Sequential(
            nn.BatchNorm1d(1001),
            nn.Linear(in_features=1001, out_features=4000),
            nn.LeakyReLU(),
            nn.BatchNorm1d(4000),
            nn.Linear(in_features=4000, out_features=2000),
            nn.LeakyReLU(),
            nn.BatchNorm1d(2000),
            nn.Linear(in_features=2000, out_features=2000),
            nn.LeakyReLU(),
            nn.BatchNorm1d(2000),
            nn.Linear(in_features=2000, out_features=1000),
            nn.LeakyReLU(),
            nn.BatchNorm1d(1000),
            nn.Linear(in_features=1000, out_features=500),
            nn.LeakyReLU(),
            nn.BatchNorm1d(500),
            nn.Linear(in_features=500, out_features=500),
            nn.LeakyReLU(),
            nn.BatchNorm1d(500),
            nn.Linear(in_features=500, out_features=250),
            nn.LeakyReLU(),
            nn.BatchNorm1d(250),
            nn.Linear(in_features=250, out_features=250),
            nn.LeakyReLU(),
            nn.BatchNorm1d(250),
            nn.Linear(in_features=250, out_features=self.n_elements),
            nn.Dropout(p=0.05),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.net(x)

In [13]:
def train_epoch(
    model: nn.Module,
    dataloader: DataLoader,
    optimizer: Optimizer,
    loss_fn: nn.Module,
    scheduler: LRScheduler = None,
) -> float:
    model.train()
    total_loss = 0
    for spectra, target in tqdm(dataloader, desc="Training"):
        optimizer.zero_grad()
        pred = model(spectra)
        loss = loss_fn(pred, target)

        total_loss += loss.item()

        loss.backward()
        # nn.utils.clip_grad_norm_(model.parameters(), 10)
        optimizer.step()

        if scheduler is not None:
            scheduler.step()

    return total_loss / len(dataloader)


@torch.no_grad()
def val_epoch(
    model: nn.Module, dataloader: DataLoader, loss_fn: nn.Module
) -> float:
    model.eval()
    loss = 0
    for spectra, target in tqdm(dataloader, desc="Validating"):

        pred = model(spectra)
        loss += float(loss_fn(pred, target).item())

    return loss / len(dataloader)

In [14]:
def train(
    model: nn.Module,
    train_loader: DataLoader,
    val_loader: DataLoader,
    optimizer: Optimizer,
    n_epochs: int,
    loss_fn: nn.Module = nn.CrossEntropyLoss(reduction="mean", label_smoothing=0.1),
    scheduler: LRScheduler = None,
) -> None:

    
    for i in range(1, n_epochs + 1):
        train_loss = train_epoch(
            model, train_loader, optimizer, loss_fn, scheduler
        )

        val_loss = val_epoch(model, val_loader, loss_fn)

        print(f"Epoch {i}: \n   Train loss: {train_loss:.5f}    |   Val loss: {val_loss:.5f}\n")

In [15]:
train_df, test_df = train_test_split(df, test_size=0.15, random_state=42)

In [16]:
batch_size = 2048
n_epochs = 30
lr = 1e-4

In [17]:
train_dataset = CustomSpectraDataset(train_df)
val_dataset = CustomSpectraDataset(test_df)

In [18]:
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
)
val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
)

In [19]:
# torch.autograd.set_detect_anomaly(True)

In [21]:
net = THzFCNN(train_dataset[0][1].shape[0])
net.to(device)
optimizer = Adam(net.parameters(), lr=lr)


print(device)

train(net, train_loader, val_loader, optimizer, n_epochs, loss_fn=nn.BCELoss(reduction="mean"))

cuda:0


Training:   0%|          | 0/77 [00:05<?, ?it/s]


RuntimeError: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
for param in net.parameters():
    print(torch.isnan(param).any())


tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
